In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv
import lxml

# Regex
regex_url = r"(http(s?):\/\/(www)?[A-Za-z0-9-?\/\.]+)"
regex_hastags = r"(#[A-Za-z0-9]{2,})"

# File url
file_url = '../datasets/goodhealth.txt'

# Basic list
csv_header = ['tweetid','date','title']

# Support scripts
%run -i scripts.py

In [2]:
# -------------------------------------------------
frame = pd.read_table(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame.head()

,tweetid,date,title
0,577117109701967872,Sun Mar 15 14:40:15 +0000 2015,It’s not hard to get a little cardio in at hom...
1,577083140977651712,Sun Mar 15 12:25:17 +0000 2015,Beauty really does come from within. Discover ...
2,577040308669669376,Sun Mar 15 09:35:05 +0000 2015,Busted! 15 myths and facts you really need to ...
3,577011455389757440,Sun Mar 15 07:40:25 +0000 2015,Get rid of a puffy tummy! See the 13 best (and...
4,576979914592804864,Sun Mar 15 05:35:06 +0000 2015,Pillow talk: Find the perfect pillow for your ...


In [3]:
# -------------------------------------------------
frame.shape

(7777, 3)

In [4]:
# -------------------------------------------------
frame.loc[:,'title'][0]

'It’s not hard to get a little cardio in at home! @KristinMcGee has an awesome plyometric move you can do anywhere: http://ow.ly/KiDDJ'

In [5]:
# -------------------------------------------------
# Split Url & Text
frame['url'] = frame['title'].map(lambda row: get_url(regex_url, row))
frame['title'] = frame['title'].map(split_txt_form_url)

http://ow.ly/KiDDJ
http://ow.ly/Kjrio
http://ow.ly/KiIY0
http://ow.ly/KiJeY
http://ow.ly/KiICW
http://ow.ly/KiGp3
http://ow.ly/KiDNu
http://ow.ly/KfQ7z
http://ow.ly/Kg8Fr
http://ow.ly/Kiex9
http://ow.ly/KitKZ
http://ow.ly/KcQCe
http://ow.ly/KfNFL
http://ow.ly/KiGEU
http://ow.ly/KiH14
http://ow.ly/KiGxc
http://ow.ly/KiDdR
http://ow.ly/KcRpI
http://wp.me/p2yZYB-vQz
http://wp.me/p2yZYB-vTP
http://ow.ly/KfD7v
http://wp.me/p2yZYB-vOh
http://wp.me/p2yZYB-vWf
http://wp.me/p2yZYB-vTu
http://ow.ly/KiUzS
http://ow.ly/KfYaZ
http://ow.ly/FqL0d
http://ow.ly/KfOMU
http://ow.ly/KfBLQ
http://ow.ly/KfUX0
http://ow.ly/KiauV
http://wp.me/p2yZYB-vWf
http://ow.ly/KdlOT
http://ow.ly/FqKV0
http://ow.ly/KfTHs
http://wp.me/p2yZYB-vFy
http://ow.ly/KfCpb
http://ow.ly/KfCU2
http://ow.ly/KcPY7
http://ow.ly/KfV7P
http://wp.me/p2yZYB-vNZ
http://wp.me/p2yZYB-uSc
http://pubx.ch/1E5Gxic
http://ow.ly/KcInk
http://wp.me/p2yZYB-vVo
http://wp.me/p2yZYB-vKC
http://wp.me/p2yZYB-vSr
http://wp.me/p2yZYB-vHE
http://ow.ly/KcTa4


http://ow.ly/EOfA4
http://ow.ly/EO7H2
http://ow.ly/EOaLE
http://ow.ly/EO9Gk
http://ow.ly/ENUUl
http://ow.ly/EO8Ej
http://ow.ly/EObWl
http://ow.ly/ENVm4
http://ow.ly/EO2aY
http://ow.ly/EObye
http://ow.ly/EO8wj
http://ow.ly/EOb8J
http://ow.ly/EO7wC
http://ow.ly/EOaXq
http://ow.ly/EObhA
http://ow.ly/EO2UM
http://ow.ly/ES3n2
http://ow.ly/ENXu3
http://ow.ly/EO0S8
http://ow.ly/ENY2p
http://ow.ly/EO8lV
http://ow.ly/EO80A
http://ow.ly/EO3kt
http://ow.ly/EO7ei
http://ow.ly/EO3FN
http://ow.ly/EO00f
http://ow.ly/EO2sD
http://ow.ly/ED0bh
http://ow.ly/Et3Z1
http://ow.ly/ED0rV
http://ow.ly/ERQE8
http://ow.ly/ENxTB
http://ow.ly/ENZJt
http://ow.ly/EbvQ7
http://ow.ly/EO0jA
http://ow.ly/ENT54
http://ow.ly/EbvMZ
http://ow.ly/ENYoQ
http://ow.ly/ENWTQ
http://ow.ly/EOZVI
http://ow.ly/ENY9E
http://ow.ly/ENTl5
http://ow.ly/ENXQ0
http://ow.ly/ECZnL
http://ow.ly/ECZRU
http://ow.ly/ECWTm
http://ow.ly/ENQ6W
http://ow.ly/ENQDJ
http://ow.ly/ENG7U
http://ow.ly/ENxBc
http://ow.ly/ECZtw
http://ow.ly/Et4ls
http://ow.ly

http://ow.ly/k7TMC
http://ow.ly/kakyD
http://ow.ly/ka9Kw
http://ow.ly/k9NUL
http://ow.ly/k7GCk
http://wp.me/p2yZYB-kQ6
http://ow.ly/k9z4l
http://wp.me/p2yZYB-kQ7
http://wp.me/p2yZYB-kQv
http://wp.me/p2yZYB-kPr
http://wp.me/p2yZYB-klK
http://ow.ly/k9MZ4
http://wp.me/p2yZYB-kPJ
http://wp.me/p2yZYB-kPg
http://ow.ly/k9jaS
http://ow.ly/k7GLR
http://ow.ly/k7YzI
http://huff.to/1531oG5
http://ow.ly/k7T3y
http://ow.ly/k7Dm4
http://ow.ly/k70FH
http://ow.ly/k7CQF
http://ow.ly/k7IoI
http://wp.me/p2yZYB-kNd
http://wp.me/p2yZYB-kNE
http://wp.me/p2yZYB-kOw
http://wp.me/p2yZYB-kND
http://ow.ly/k7ncv
http://bit.ly/119AiJB
http://ow.ly/k714u
http://ow.ly/k7lqA
http://ow.ly/k74tD
http://ow.ly/k70RR
http://wp.me/p2yZYB-kNs
http://wp.me/p2yZYB-kLZ
http://ow.ly/k5y0E
http://bit.ly/12M4RTS
http://ow.ly/k5ftz
http://ow.ly/k54h9
http://wp.me/p2yZYB-kMx
http://wp.me/p2yZYB-kMt
http://wp.me/p2yZYB-kMb
http://ow.ly/k53ec
http://wp.me/p2yZYB-krt
http://ow.ly/k4RCw
http://ow.ly/k4LCD
http://ow.ly/k4DKL
http://ow.ly

http://wp.me/p2yZYB-hzR
http://wp.me/p2yZYB-hzt
http://bit.ly/bPW2Tk
http://pinterest.com/goodhealth/breast-cancer-awareness/
http://wp.me/p2yZYB-hzG
http://wp.me/p2yZYB-hoG
http://wp.me/p2yZYB-hm6
http://wp.me/p2yZYB-hpI
http://bit.ly/vpamTf
http://wp.me/p2yZYB-hmb
http://ow.ly/e0IBu
http://wp.me/p2yZYB-hof
http://ow.ly/dZhG4
http://wp.me/p2yZYB-ho8
http://ow.ly/dZh0f
http://wp.me/p2yZYB-hoj
http://wp.me/p2yZYB-hmR
http://wp.me/p2yZYB-hnj
http://wp.me/p2yZYB-hlO
http://wp.me/p2yZYB-hlP
http://wp.me/p2yZYB-hl2
http://wp.me/p2yZYB-hlE
http://wp.me/p2yZYB-hla
http://wp.me/p2yZYB-hf3
http://wp.me/p2yZYB-hjq
http://wp.me/p2yZYB-hkJ
http://ow.ly/dS90O
http://ow.ly/dS8bs
http://ow.ly/dRWPo
http://wp.me/p2yZYB-hkB
http://wp.me/p2yZYB-hjx
http://trib.al/Z2xEPw
http://wp.me/p2yZYB-hjj
http://bit.ly/bgJqiL
http://wp.me/p2yZYB-hgo
http://wp.me/p2yZYB-hd1
http://wp.me/p2yZYB-hcy
http://wp.me/p2yZYB-hgf
http://on.fb.me/vhBkfB
http://ow.ly/dGAzP
http://ow.ly/dMpmZ
http://ow.ly/dMcDE
http://wp.me/p2y

http://bit.ly/kAaWhU
http://ow.ly/5kl4A
http://bit.ly/jwEUx3
http://on-msn.com/mSVmVK
http://bit.ly/cROFSy
http://bit.ly/ikCWNV
http://bit.ly/j59KMJ
http://bit.ly/gotWKb
http://bit.ly/kxgWC0
http://bit.ly/iO7SHP
http://bit.ly/mQ13Qh
http://bit.ly/l6uasl
http://bit.ly/m4A7WF
http://bit.ly/iWixjY
http://bit.ly/iM8sib
http://bit.ly/mj3azN
http://bit.ly/kGcFwO
http://bit.ly/kLmCLT
http://bit.ly/cKsJBo
http://bit.ly/ix0Byp
http://bit.ly/mAyM5S
http://bit.ly/k7aU4D
http://bit.ly/cROFSy
http://bit.ly/jfFfCW
http://bit.ly/kvfTkK
http://bit.ly/jvI7pn
http://bit.ly/lthgVu
http://bit.ly/ls96AG
http://fxn.ws/lI50rw
http://bit.ly/jUXHcI
http://huff.to/jsrvOl
http://bit.ly/hTi0yz
http://bit.ly/mGHOAZ
http://aol.it/inFaPk
http://bit.ly/m5SHDl
http://wp.me/pYxcF-4gB
http://bit.ly/kIbW2H
http://bit.ly/bPPPe9
http://vitaljuice.com/ecs/13277.htm
http://bit.ly/cROFSy
http://bit.ly/9vEf5q
http://ow.ly/5hzRK
http://bit.ly/mpdOYr
http://bit.ly/jiuczb
http://bit.ly/mmRFz5
http://bit.ly/kL7QuX
http://bit.ly/gE

...

http://wp.me/p2yZYB-juT
The secret reason you’re pigging out (and what to do about it!) from RD @CynthiaSass http://wp.me/p2yZYB-jv9


http://wp.me/p2yZYB-jv9
Think you need to do crunches to get flat abs? Think again! Here are 24 Fat-Burning Exercises (No Crunches) #fitness http://ow.ly/h5TVs


http://ow.ly/h5TVs
Good morning! Can't seem to wake up your eyes? Try these Makeup Tips to Make Your Eyes Pop: http://ow.ly/h4GbY #beauty


http://ow.ly/h4GbY
Omega #Omega! High-protein, heart-healthy meals are on the menu! Here are 20 SIMPLE, DELICIOUS #SALMON #RECIPES http://ow.ly/gVSrn


http://ow.ly/gVSrn
Is your sex life grand or bland? Follow these Daily Tips for More Satisfying Sex. http://ow.ly/h4EHK


http://ow.ly/h4EHK
3 ways to feel happier this year is no sweat for cover star @jenniegarth. Hear how she tackles it all on her blog on http://Health.com…


http://Health.com
Renewing healthy choices every day is easy for @jenniegarth. She shares all in our Jan/Feb issue. http://wp.me/p2y



http://wp.me/p2yZYB-i9c
Why We All Need A "Pre-#Diabetes" Diet http://wp.me/p2yZYB-i8z


http://wp.me/p2yZYB-i8z
3 Tasty Ways to Eat Spaghetti #Squash http://wp.me/p2yZYB-hW1


http://wp.me/p2yZYB-hW1
5 Sneaky Snacks That Are Making You Fat http://wp.me/p2yZYB-i8i


http://wp.me/p2yZYB-i8i
New Treatment For Head Lice? http://wp.me/p2yZYB-i9l


http://wp.me/p2yZYB-i9l
Healthy Recipes to Weather #Hurricane #Sandy http://wp.me/p2yZYB-i7J


http://wp.me/p2yZYB-i7J
Ultimate Beauty Tool: 7 Ways to Use Q-Tips http://wp.me/p2yZYB-i4I


http://wp.me/p2yZYB-i4I
Flight canceled due to #Hurricane #Sandy? Here's how to work out in your hotel room, no gym necessary http://wp.me/p2yZYB-hVf


http://wp.me/p2yZYB-hVf
Living Well with Psoriasis on World Psoriasis Day http://wp.me/p2yZYB-i4Z


http://wp.me/p2yZYB-i4Z
30 Foods Under 40 Calories http://ow.ly/eLxpD


http://ow.ly/eLxpD
It's National Pumpkin Day! Our 12 Favorite Pinterest Pins http://wp.me/p2yZYB-i5m http://wp.me/s2yZYB-pumpkin


http://wp


http://ow.ly/apgGl
RT @kamila_kaminska: Win a luxury trip for 2 to the Palmetto Bay Plantation in Honduras from @PureWow @TimeoutNY & @goodhealth! Enter...


@takeyausa sure did! http://bit.ly/oATjMY


http://bit.ly/oATjMY
RT @enjoyyourlifes: @goodhealth Ideal Distance Between Pregnancy http://www.lifewithstyle.witingtresno.com/ideal-inter-pregnancy-timeline/healthy.php


http://www.lifewithstyle.witingtresno.com/ideal-inter-pregnancy-timeline/healthy.php
RT @johnsonthompson: “@goodhealth: When Eating Healthy Turns Obsessive http://www.health.com/health/article/0,,20556862,00.html” @samanthaafraser lol


http://www.health.com/health/article/0
http://pinterest.com/pin/34832597088225426/ The Top Power Foods for You


http://pinterest.com/pin/34832597088225426/
The Top Power Foods for You http://pinterest.com/pin/34832597088225426/


http://pinterest.com/pin/34832597088225426/
When Eating Healthy Turns Obsessive http://www.health.com/health/article/0,,20556862,00.html


http://www.health



We desperately need a test that does work in younger women -@DrSusanLove on mammograms. Tune in here: on.fb.me/tq9359 Tweet your questions


Stress reduction has been shown to reduce #breastcancer --@DrSusanLove Tune in here: on.fb.me/tq9359


There's no data to suggest that having IVF increases #breastcancer risk -@DrSusanLove Tune in here: on.fb.me/tq9359


We're getting started, come join us! RT @DrSusanLove: TODAY live video #breastcancer chat at 1pm EST. Tune in here: http://on.fb.me/tq9359


http://on.fb.me/tq9359
13 Easy #Halloween Party Recipes from our friends at @Real_Simple: http://bit.ly/voy631


http://bit.ly/voy631
Join @goodhealth and @ArmyofWomen for a live chat w/ @DrSusanLove today at 1pm EST. Submit questions & tune in here: http://tinyurl.com/6d5jgy3


http://tinyurl.com/6d5jgy3
RT @TIMEHealthland: Is black licorice a trick or a treat? The FDA weighs in 


@MelisaNutrition Love it! RT: @goodhealth One #Halloween I was "80s Workout Girl". http://bit.ly/txbKHe


htt


http://bit.ly/ix0Byp
Who's planning on doing some cooking this weekend? Tell us what you're thinking of making and we'll tweet you a lightened-up recipe!


#FoodieFriday: A fro-yo with built-in portion control! http://bit.ly/mAyM5S


http://bit.ly/mAyM5S
Weekly Picks: 100 Recipes for Under $1, Birth Control for Men, and 10 Ways to Snack in the Summer: http://bit.ly/k7aU4D


http://bit.ly/k7aU4D
@MyBlissPoint You're welcome! Thanks for being a fan!


@MyBlissPoint Sorry you feel that way! We're working on it!


#QuickPick: What's sabotaging your diet at work? Vote here: http://bit.ly/cROFSy And tell us: How do you avoid overdoing it?


http://bit.ly/cROFSy
@sajohnson015 :-) Thanks for being a fan!


Americans love fast food, but we don't all love it equally. Here, 10 states that eat the most: http://bit.ly/jfFfCW Did yours make the list?


http://bit.ly/jfFfCW
Eat this and burn more fat: http://bit.ly/kvfTkK


http://bit.ly/kvfTkK
RT @fgfitness: #FitnessFriday: This terrific new workou

In [6]:
frame.head()

,tweetid,date,title,url
0,577117109701967872,Sun Mar 15 14:40:15 +0000 2015,It’s not hard to get a little cardio in at hom...,http://ow.ly/KiDDJ
1,577083140977651712,Sun Mar 15 12:25:17 +0000 2015,Beauty really does come from within. Discover ...,http://ow.ly/Kjrio
2,577040308669669376,Sun Mar 15 09:35:05 +0000 2015,Busted! 15 myths and facts you really need to ...,http://ow.ly/KiIY0
3,577011455389757440,Sun Mar 15 07:40:25 +0000 2015,Get rid of a puffy tummy! See the 13 best (and...,http://ow.ly/KiJeY
4,576979914592804864,Sun Mar 15 05:35:06 +0000 2015,Pillow talk: Find the perfect pillow for your ...,http://ow.ly/KiICW


In [7]:
# -------------------------------------------------
# Scrapt form web
df_temp = pd.DataFrame()
frame_urls = frame['url'].values

In [8]:
# -------------------------------------------------
# Run for this jupyter notebook
title_attempt = True
for key, url in enumerate(frame_urls):
    try:
        # Start the connection
        res = requests.get(url)        

        # Check return header
        if res.status_code == 200:
            print("Header:  ", 200)
            print(url)
            
            # Initialize XPath
            xpath_selector = Selector(text=res.text)
            
            list_content = dict()
            
            # Scrapt Text with XPath
            contents = xpath_selector.xpath('//div[@class="glide-slide-desc"]/p/text()').extract()
            contents = join_txt(contents)
            print(len(contents))
            # Tags
#             tags = xpath_selector.xpath('//div[@class="tags"]/ul/li/a/text()').extract()     
#             print(len(tags))
            # Author
            author = xpath_selector.xpath('//a[@class="bold author-name"]/text()').extract()
            if len(author):
                author = author[0]
            else:
                author = ''
            print(len(author))
            list_content.update({'contents': contents,'tags': '','author': author})
            
            if title_attempt:
                print("TItle: ", title_attempt)
                df_temp = pd.DataFrame(columns = list(list_content.keys()))
                frame = pd.concat([frame, df_temp])
                print(frame.columns)                
                title_attempt = False
                
            for k_, v_ in list_content.items():
                print(k_," : " ,len(v_))
                frame.loc[key,k_] = v_
            print("==================\n")

        else:
            print("['ERRORS'] {} , at {}".format(res.status_code, url))
    except:
        pass
    

Header:   200
http://ow.ly/KiDDJ
0
0
TItle:  True
Index(['author', 'contents', 'date', 'tags', 'title', 'tweetid', 'url'], dtype='object')
contents  :  0
tags  :  0
author  :  0



scripts.py:38: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  # Find all possible hastags


Header:   200
http://ow.ly/KiIY0
488
16
contents  :  488
tags  :  0
author  :  16

Header:   200
http://ow.ly/KiJeY
7879
21
contents  :  7879
tags  :  0
author  :  21

Header:   200
http://ow.ly/KiICW
2209
17
contents  :  2209
tags  :  0
author  :  17

Header:   200
http://ow.ly/KiGp3
7829
13
contents  :  7829
tags  :  0
author  :  13

Header:   200
http://ow.ly/KiDNu
6073
17
contents  :  6073
tags  :  0
author  :  17

Header:   200
http://ow.ly/KfQ7z
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://ow.ly/Kg8Fr
509
13
contents  :  509
tags  :  0
author  :  13

Header:   200
http://ow.ly/Kiex9
0
11
contents  :  0
tags  :  0
author  :  11

Header:   200
http://ow.ly/KitKZ
8611
16
contents  :  8611
tags  :  0
author  :  16

Header:   200
http://ow.ly/KcQCe
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://ow.ly/KfNFL
2133
12
contents  :  2133
tags  :  0
author  :  12

Header:   200
http://ow.ly/KiGEU
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200


['ERRORS'] 410 , at http://wp.me/p2yZYB-vOo
Header:   200
http://ow.ly/K6y7r
4292
0
contents  :  4292
tags  :  0
author  :  0

Header:   200
http://ow.ly/FqKaH
0
0
contents  :  0
tags  :  0
author  :  0

['ERRORS'] 410 , at http://wp.me/p2yZYB-vCC
Header:   200
http://ow.ly/K6zWY
7763
16
contents  :  7763
tags  :  0
author  :  16

Header:   200
http://ow.ly/K6yHQ
8041
12
contents  :  8041
tags  :  0
author  :  12

Header:   200
http://ow.ly/K6Bx9
509
13
contents  :  509
tags  :  0
author  :  13

Header:   200
http://ow.ly/K6Du6
0
11
contents  :  0
tags  :  0
author  :  11

['ERRORS'] 410 , at http://wp.me/p2yZYB-vOP
Header:   200
http://ow.ly/K6wu8
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://ow.ly/K6x3x
11756
16
contents  :  11756
tags  :  0
author  :  16

Header:   200
http://ow.ly/K6C1s
1212
14
contents  :  1212
tags  :  0
author  :  14

Header:   200
http://ow.ly/K6CsV
0
16
contents  :  0
tags  :  0
author  :  16

Header:   200
http://ow.ly/K6yom
8159
15
content

Header:   200
http://ow.ly/JZB14
0
0
contents  :  0
tags  :  0
author  :  0

['ERRORS'] 410 , at http://wp.me/p2yZYB-vKH
Header:   200
http://ow.ly/JVSnn
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://ow.ly/JVLGY
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://ow.ly/JVS99
2641
14
contents  :  2641
tags  :  0
author  :  14

['ERRORS'] 410 , at http://wp.me/p2yZYB-vKb
Header:   200
http://ow.ly/JVPQx
7324
0
contents  :  7324
tags  :  0
author  :  0

Header:   200
http://ow.ly/JVIeU
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://ow.ly/JVGqd
2298
13
contents  :  2298
tags  :  0
author  :  13

Header:   200
http://ow.ly/JVI2z
594
0
contents  :  594
tags  :  0
author  :  0

Header:   200
http://ow.ly/JVMCZ
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://ow.ly/JVLt2
2209
17
contents  :  2209
tags  :  0
author  :  17

['ERRORS'] 410 , at http://wp.me/p2yZYB-tP5
['ERRORS'] 410 , at http://wp.me/p2yZYB-vmt
['ERRORS'] 410 , at 

...

Header:   200
http://bit.ly/pBX19m
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://bit.ly/n8nNEb
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://bit.ly/r6Jw8z
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://bit.ly/qv0mNv
0
21
contents  :  0
tags  :  0
author  :  21

Header:   200
http://bit.ly/rsgwqA
0
11
contents  :  0
tags  :  0
author  :  11

Header:   200
http://bit.ly/nBwed1
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://bit.ly/qboofq
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://ow.ly/6smbd
47207
18
contents  :  47207
tags  :  0
author  :  18

Header:   200
http://nyti.ms/oTwpHg
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://bit.ly/qHthk8
0
12
contents  :  0
tags  :  0
author  :  12

Header:   200
http://bit.ly/oPdFFX
0
11
contents  :  0
tags  :  0
author  :  11

Header:   200
http://bit.ly/pMULvZ
0
12
contents  :  0
tags  :  0
author  :  12

Header:   200
http://bit.ly/pjFZc

Header:   200
http://bit.ly/oJnuAV
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://bit.ly/pfNSyM
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://bit.ly/cROFSO
1127
0
contents  :  1127
tags  :  0
author  :  0

['ERRORS'] 403 , at http://bit.ly/qhnukA
['ERRORS'] 410 , at http://wp.me/piR1W-cBC
Header:   200
http://bit.ly/onpb6h
0
14
contents  :  0
tags  :  0
author  :  14

['ERRORS'] 403 , at http://bit.ly/qeR3BM
Header:   200
http://bit.ly/nbfEBY
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://ow.ly/5W93e
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://bit.ly/cROFSO
1127
0
contents  :  1127
tags  :  0
author  :  0

Header:   200
http://bit.ly/mUMr5d
1213
13
contents  :  1213
tags  :  0
author  :  13

Header:   200
http://bit.ly/oiLaB6
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://bit.ly/r26ppX
0
12
contents  :  0
tags  :  0
author  :  12

['ERRORS'] 403 , at http://bit.ly/oDINyU
Header:   200
http://b

Header:   200
http://bit.ly/qztvT4
448
16
contents  :  448
tags  :  0
author  :  16

Header:   200
http://bit.ly/mwGaJq
0
15
contents  :  0
tags  :  0
author  :  15

['ERRORS'] 404 , at http://ow.ly/5L552
Header:   200
http://bit.ly/ql4sAj
0
16
contents  :  0
tags  :  0
author  :  16

['ERRORS'] 404 , at http://bit.ly/oXgHgI
Header:   200
http://bit.ly/q6ocYT
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://bit.ly/b3rUgu
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://bit.ly/qffhRl
1914
0
contents  :  1914
tags  :  0
author  :  0

['ERRORS'] 403 , at http://bit.ly/rtamiY
Header:   200
http://bit.ly/oZPXtU
0
0
contents  :  0
tags  :  0
author  :  0

['ERRORS'] 403 , at http://bit.ly/pvuZjX
Header:   200
http://bit.ly/o1dgbk
0
0
contents  :  0
tags  :  0
author  :  0

['ERRORS'] 403 , at http://bit.ly/oZkqtS
Header:   200
http://abcn.ws/q4fqM8
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://bit.ly/fOXhsZ
0
0
contents  :  0
tags  :  0
auth

Header:   200
http://fitsu.gr/18106283
0
0
contents  :  0
tags  :  0
author  :  0

['ERRORS'] 403 , at http://bit.ly/k7hUB7
Header:   200
http://ow.ly/5uPgK
8197
0
contents  :  8197
tags  :  0
author  :  0

Header:   200
http://bit.ly/cROFSy
1127
0
contents  :  1127
tags  :  0
author  :  0

Header:   200
http://bit.ly/jp3RpI
0
11
contents  :  0
tags  :  0
author  :  11

['ERRORS'] 404 , at http://bit.ly/eoQvtT
Header:   200
http://bit.ly/ixhQvU
0
0
contents  :  0
tags  :  0
author  :  0

['ERRORS'] 403 , at http://bit.ly/aJFGz7
Header:   200
http://bit.ly/cROFSy
1127
0
contents  :  1127
tags  :  0
author  :  0

['ERRORS'] 403 , at http://bit.ly/lhwP9A
Header:   200
http://bit.ly/mBqJCw
5877
15
contents  :  5877
tags  :  0
author  :  15

Header:   200
http://bit.ly/lKhSM4
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://bit.ly/lzf0Z4
0
15
contents  :  0
tags  :  0
author  :  15

Header:   200
http://ow.ly/5ukPH
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
ht

Header:   200
http://bit.ly/mj3azN
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://bit.ly/kGcFwO
0
8
contents  :  0
tags  :  0
author  :  8

Header:   200
http://bit.ly/kLmCLT
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://bit.ly/cKsJBo
675
0
contents  :  675
tags  :  0
author  :  0

Header:   200
http://bit.ly/ix0Byp
0
11
contents  :  0
tags  :  0
author  :  11

Header:   200
http://bit.ly/mAyM5S
0
14
contents  :  0
tags  :  0
author  :  14

Header:   200
http://bit.ly/k7aU4D
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://bit.ly/cROFSy
1127
0
contents  :  1127
tags  :  0
author  :  0

Header:   200
http://bit.ly/jfFfCW
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://bit.ly/kvfTkK
0
12
contents  :  0
tags  :  0
author  :  12

['ERRORS'] 403 , at http://bit.ly/jvI7pn
Header:   200
http://bit.ly/lthgVu
0
13
contents  :  0
tags  :  0
author  :  13

Header:   200
http://bit.ly/ls96AG
270
0
contents  :  270
tags  :  0
aut

In [9]:
# -------------------------------------------------
frame.head()

,author,contents,date,tags,title,tweetid,url
0,,,Sun Mar 15 14:40:15 +0000 2015,,It’s not hard to get a little cardio in at hom...,5.771171e+17,http://ow.ly/KiDDJ
1,NaN,NaN,Sun Mar 15 12:25:17 +0000 2015,NaN,Beauty really does come from within. Discover ...,5.770831e+17,http://ow.ly/Kjrio
2,Amanda MacMillan,? You're not alone: The cosmetic condition aff...,Sun Mar 15 09:35:05 +0000 2015,,Busted! 15 myths and facts you really need to ...,5.770403e+17,http://ow.ly/KiIY0
3,Charlene K. Petitjean,Let's talk about something uncomfortable: gas ...,Sun Mar 15 07:40:25 +0000 2015,,Get rid of a puffy tummy! See the 13 best (and...,5.770115e+17,http://ow.ly/KiJeY
4,Elizabeth Roehrig,Not getting enough rest? The problem could be ...,Sun Mar 15 05:35:06 +0000 2015,,Pillow talk: Find the perfect pillow for your ...,5.769799e+17,http://ow.ly/KiICW


In [10]:
# -------------------------------------------------
frame.shape

(7777, 7)

In [11]:
frame.to_csv('./run_data/goodhealth_scrapt_v.csv', sep='|', encoding='utf-8', index=False)